<a href="https://colab.research.google.com/github/adigenova/uohdb/blob/main/code/Arango_GraphDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ArangoDB Trabajando con grafos

ArangoDB es una base de datos multimodelo  con modelos de datos flexibles para documentos, grafos y clave/valor. Posee el lenguaje AQL que es similar al SQL para realizar consultas.

# Configuración

Antes de comenzar con ArangoDB, debemos preparar nuestro entorno y crear una base de datos temporal en el Servicio Oasis.

In [1]:
%%capture
!git clone https://github.com/joerg84/ArangoDBUniversity.git
!rsync -av ArangoDBUniversity/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [2]:
import json
import requests
import sys
import oasis
import time
from IPython.display import JSON

from arango import ArangoClient

Creamos una base de datos temporal

In [3]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="Pregel", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
db = oasis.connect_python_arango(login)

Requesting new temp credentials.
Temp database ready to use.


In [4]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

https://tutorials.arangodb.cloud:8529
Username: TUThggru76jgk0t133fibhv5
Password: TUTchbwtq7n2b6j5yr3wmuvxp
Database: TUTqp19jvaazh7txkel4fr1u


Podemos usar la URL anterior para revisar nuestra base de datos arango temporal.



# Cargando datos en arangoDB

In [ ]:
! wget https://github.com/adigenova/uohpmd/blob/main/data/nosql/aeropuertos.csv.gz\?raw=true -O aeropuertos.csv.gz
! wget https://github.com/adigenova/uohpmd/blob/main/data/nosql/vuelos.csv.gz\?raw=true -O vuelos.csv.gz
! zcat /content/aeropuertos.csv.gz | wc -l
! zcat /content/vuelos.csv.gz | wc -l
! zcat /content/aeropuertos.csv.gz > /content/aeropuertos.csv
! zcat /content/vuelos.csv.gz > /content/vuelos.csv
! echo ":::AEROPUERTOS:::"
! head /content/aeropuertos.csv
! echo ":::VUELOS:::"
! head /content/vuelos.csv

In [ ]:
# cargamos los aeropuertos A LA db
! ./tools/arangoimport  -c none --server.endpoint http+ssl://{login["hostname"]}:{login["port"]} --server.username {login["username"]} --server.database {login["dbName"]} --server.password {login["password"]} --file /content/aeropuertos.csv --collection airports --create-collection true --type csv

In [ ]:
# CARGAMOS los vuelos a la DB
! ./tools/arangoimport  -c none --server.endpoint http+ssl://{login["hostname"]}:{login["port"]} --server.username {login["username"]} --server.database {login["dbName"]} --server.password {login["password"]} --file /content/vuelos.csv --collection flights --create-collection true --type csv --create-collection-type edge

In [12]:
aql = db.aql

In [ ]:
#Determine los nombres de todos los aeropuertos a los que se puede llegar directamente (1 paso)
# desde LAX.
query = """
WITH airports
FOR a IN 1..1 OUTBOUND 'airports/LAX' flights
RETURN DISTINCT [a._key,a.name,a.city]
"""
query_result = aql.execute(query)
for doc in  query_result:
 print(doc)

In [ ]:
# Listar todos l
query = """
WITH airports
FOR airport, flight IN ANY 'airports/LAX' flights
FILTER flight.Month == 1 AND flight.Day >= 5 AND flight.Day <= 7
RETURN { city: airport.city, time: flight.ArrTimeUTC }
"""
query_result = aql.execute(query)
for doc in  query_result:
 print(doc)

In [ ]:
# Listar todos los vuelos del avion N238JB
query = """
FOR flight IN flights
FILTER flight.TailNum == "N238JB"
RETURN flight
"""
query_result = aql.execute(query)
for doc in  query_result:
 print(doc)


In [ ]:
# Determinar todos los aeropuertos directamente accesibles desde LAX
query = """
WITH airports
FOR airport IN OUTBOUND 'airports/LAX' flights
 OPTIONS { bfs: true, uniqueVertices: 'global' }
  RETURN airport
"""
query_result = aql.execute(query)
for doc in  query_result:
 print(doc)


In [ ]:
# Encuentre la ruta más corta entre el aeropuerto BIS y el aeropuerto JFK y
# devuelva los nombres de los aeropuertos en la ruta.

query = """
WITH airports
FOR v IN OUTBOUND SHORTEST_PATH 'airports/BIS' TO 'airports/JFK' flights RETURN v.name
"""
query_result = aql.execute(query)
for doc in  query_result:
 print(doc)
